***GENERATED CODE FOR descriptiveanalyticsml PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
from sklearn.cluster import KMeans
import optuna
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()

    def objective(trial):
        n_clusters = int(trial.suggest_loguniform("n_clusters", 2, 10))
        classifier_obj = KMeans(n_clusters=n_clusters).fit(pd_df)
        labels = classifier_obj.predict(pd_df)
        score = metrics.silhouette_score(pd_df, labels, metric='euclidean')
        return score
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    best = study.best_params
    best_param = int(best["n_clusters"])
    model = KMeans(n_clusters=best_param).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        autokmeans_obj = kmeans(df=spark_DF, TrainingPercent=trainingPercent)
        [autokmeans_obj]


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run descriptiveanalyticsmlHooks.ipynb
try:
	#sourcePreExecutionHook()

	datasource = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/part-00000-7561895a-9940-4f80-a05b-28ed5d9ce0d7-c000.csv', 'filename': '1712425505CleanedData.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'FilePath': '/Global SuperStore Sales 2016/CleanedData.csv', 'viewFileName': 'CleanedData.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run descriptiveanalyticsmlHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofedatan = TransformationMain.run(datasource,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Postal Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "41032", "mean": "55240.66", "stddev": "14019.88", "min": "1453", "max": "99301", "missing": "0"}, "updatedLabel": "Postal Code"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Profit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "31.21", "stddev": "185.19", "min": "-6599.0", "max": "8399.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Profit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Shipping Cost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "26.82", "stddev": "61.3", "min": "-292.0", "max": "933.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Shipping Cost"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Order ID_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "8562.42", "stddev": "7249.54", "min": "0.0", "max": "25727.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Order ID_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Order Date_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "450.09", "stddev": "345.65", "min": "0.0", "max": "1429.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Order Date_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Date_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "516.59", "stddev": "387.54", "min": "0.0", "max": "1463.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Date_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "41032", "mean": "0.65", "stddev": "0.91", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "Ship Mode_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer ID_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "5218.72", "stddev": "4534.92", "min": "0.0", "max": "17413.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer ID_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer Name_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "350.95", "stddev": "224.64", "min": "0.0", "max": "795.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Customer Name_stringindex..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Segment_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "41032", "mean": "0.66", "stddev": "0.77", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Segment_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "City_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "599.24", "stddev": "766.46", "min": "0.0", "max": "3649.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "City_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "State_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "147.47", "stddev": "194.92", "min": "0.0", "max": "1101.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "State_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Country_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "16.4", "stddev": "22.37", "min": "0.0", "max": "164.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Country_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "7.02", "stddev": "5.7", "min": "0.0", "max": "22.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Market_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "1.59", "stddev": "1.32", "min": "0.0", "max": "4.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Market_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product ID_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "1057.6", "stddev": "899.22", "min": "0.0", "max": "3787.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Product ID_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Category_stringindexer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "41032", "mean": "0.59", "stddev": "0.8", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Category_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sub-Category_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "6.01", "stddev": "4.67", "min": "0.0", "max": "16.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sub-Category_stringindexe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Product Name_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "1062.76", "stddev": "901.18", "min": "0.0", "max": "3784.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Product Name_stringindexe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sales_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "6443.65", "stddev": "6589.26", "min": "0.0", "max": "22954.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sales_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Quantity_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "2.98", "stddev": "9.32", "min": "0.0", "max": "230.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Quantity_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Discount_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "2.12", "stddev": "3.98", "min": "0.0", "max": "49.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Discount_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Order Priority_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "1.13", "stddev": "8.88", "min": "0.0", "max": "226.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Order Priority_stringinde..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Returned_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "0.01", "stddev": "0.32", "min": "0.0", "max": "22.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Returned_stringindexer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Person_stringindexer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "41032", "mean": "7.06", "stddev": "5.76", "min": "0.0", "max": "36.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Person_stringindexer"}]}))

	#transformationPostExecutionHook(autofedatan)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run descriptiveanalyticsmlHooks.ipynb
try:
	#mlPreExecutionHook()

	clusteringml = Clustering.run(autofedatan,spark,json.dumps( {"autoClustering": 1, "originalfile": "/FileStore/platform/uploadedSourceFiles/part-00000-7561895a-9940-4f80-a05b-28ed5d9ce0d7-c000.csv", "model": "Kmeans", "TrainingPercent": 100, "totalClusters": 2}))

	#mlPostExecutionHook(clusteringml)

except Exception as ex: 
	logging.error(ex)
#spark.stop()
